In [ ]:
# !pip install crewai
# !pip install 'crewai[tools]'
# !pip install langchain-groq

# **LIBS**

In [1]:
from crewai import Agent, Task, Crew, Process
from crewai_tools import tool, FileReadTool
from langchain_groq import ChatGroq

api_key="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"


# **TOOLS**

In [2]:
# Criando as ferramentas
file_read_tool = FileReadTool(file_path='projetos\demanda_consultar_cep.txt')

# **LLM**

In [3]:
llm_llama=ChatGroq(temperature=0,
             model_name="llama3-70b-8192",
             api_key=api_key)

# **AGENTS**

In [4]:
analista_requisitos = Agent(
  role="Analista de Requisitos",
  goal="Coleta, análise e documentação dos requisitos do sistema.",
  backstory=(
            "Entendimento do negócio, técnicas de elicitação de requisitos, documentação e comunicação eficaz."
  ),
  verbose=True,
  tools=[file_read_tool],
  allow_delegation=True,
  llm=llm_llama,
  max_iter=10,
  max_rpm=50,
)


desenvolvedor_backend = Agent(
  role="Desenvolvedor Back-End",
  goal="Desenvolvimento da API de consulta em C#, implementação de lógica de negócios e integração com o banco de dados.",
  backstory=(
            "Programação em C#, ASP.NET Core, Entity Framework, conhecimentos em SQL Server, práticas de desenvolvimento de API RESTful."
  ),
  verbose=True,
  allow_delegation=True,
  llm=llm_llama,
  max_iter=12,
  max_rpm=50,
  )

desenvolvedor_frontend = Agent(
  role="Desenvolvedor Front-End (Angular)",
  goal="Desenvolvimento da interface web para pesquisa de CEP, implementação de componentes de interface e comunicação com a API.",
  backstory=(
            "Programação em Angular, TypeScript, HTML, CSS, integração com APIs RESTful, boas práticas de UX/UI."
  ),
  verbose=True,
  allow_delegation=True,
  llm=llm_llama,
  max_iter=12,
  max_rpm=50,
)


administrador_bancodados = Agent(
  role="Administrador de Banco de Dados (DBA)",
  goal="Design e gerenciamento do banco de dados SQL Server, garantindo desempenho, segurança e integridade dos dados.",
  backstory=(
            "Administração de SQL Server, otimização de consultas, backup e recuperação de dados, segurança de banco de dados."
  ),
  verbose=True,
  allow_delegation=True,
  llm=llm_llama,
  max_iter=12,
  max_rpm=50,
)

# **TASKS**

In [6]:
definir_user_story = Task(
    description="Definir a user story e seus critérios de aceitação.",
    expected_output="User story com critério de aceitação",
    tools=[file_read_tool],
    agent=analista_requisitos
)

banco_dados = Task(
    description="Escrever a estrutura de banco de dados para implementar a user story.",
    expected_output='Criar tabelas, stored procedures',
    output_file='script_db.sql',
    agent=administrador_bancodados,
)

api = Task(
    description="Desenvolver a API",
    expected_output="Criar o codigo do projeto ASP.NET Core Web API",
    output_file="API_response_structure.cs",
    agent=desenvolvedor_backend,
)

front = Task(
    description="Desenvolver componentes de interface do usuário usando o framework Angular com base nos requisitos fornecidos",
    expected_output="Componentes Angular funcionais que correspondem ao requisito de interface do usuário.",
    output_file="app.component.ts",
    agent=desenvolvedor_frontend,
)

# **CREW**

In [7]:
# Formação da tripulação focada em tecnologia com configurações aprimoradas
equipe = Crew(
  agents=[analista_requisitos,
          desenvolvedor_backend,
          desenvolvedor_frontend,
          administrador_bancodados
          ],
  tasks=[definir_user_story,
         banco_dados,
         api,
         front
         ],
  process=Process.sequential,  # Execução sequencial de tarefas é padrão
  verbose = 2
)

In [8]:
resultado = equipe.kickoff()

 [DEBUG]: == Working Agent: Analista de Requisitos
 [INFO]: == Starting Task: Definir a user story e seus critérios de aceitação.


> Entering new CrewAgentExecutor chain...
Thought: I need to understand the business requirements to define the user story and its acceptance criteria. Let's start by reading the content of the file "projetos\demanda_consultar_cep.txt" to gather more information.

Action: Read a file's content
Action Input: {"file_path": "projetos\demanda_consultar_cep.txt"} 

DocumentaÃ§Ã£o de Projeto: Consulta de CEP

VisÃ£o Geral
Este projeto tem como objetivo desenvolver uma aplicaÃ§Ã£o completa para consulta de CEPs, que 
retorna o endereÃ§o completo associado. A soluÃ§Ã£o envolve trÃªs componentes principais:

    1. Banco de Dados (SQL Server)
    2. API de Consulta (C#, ASP.NET Core.). 
    3. Interface Web de Pesquisa (Angular)


Tabela SQL:
CREATE TABLE Enderecos (
    Id INT PRIMARY KEY IDENTITY(1,1),
    CEP VARCHAR(10) NOT NULL,
    Logradouro VARCHAR(255) NOT

In [ ]:
print(resultado)